In [4]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI (or Ollama)

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.  



In [5]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [6]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions

# Let's make a quick call to a ChatGPT model to get started, as a preview!

In [8]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "We are having conversation with Chat GPT for the first time"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

That's great! I'm here to help with any questions or topics you’d like to discuss. What would you like to know or talk about?


## Execute the scrapping of the Website

In [9]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [14]:
# Let's try one out. Change the website and add print statements to follow along.

websiteDescription = Website("https://apple.com")
print(websiteDescription.title)
print(websiteDescription.text)

Apple
Apple
Apple
Store
Mac
iPad
iPhone
Watch
Vision
AirPods
TV & Home
Entertainment
Accessories
Support
0
+
iPhone
Meet the iPhone 16 family.
Learn more
Shop iPhone
Built for Apple Intelligence.
Buy Mac or iPad for college
with education savings
Choose AirPods or an eligible accessory
1
Shop
F1 The Movie
Starring Brad Pitt. In theaters June 27.
Watch the trailer
Watch the trailer
Get tickets
MacBook Air
Sky blue color.
Sky high performance with M4.
Learn more
Buy
Built for Apple Intelligence.
iPad Pro
Unbelievably thin. Incredibly powerful.
Learn more
Buy
Built for Apple Intelligence.
iPad Air
Now supercharged by the M3 chip.
Learn more
Buy
Built for Apple Intelligence.
Apple Podcasts
20 years of podcasts. 20 unforgettable shows.
Explore the list
Apple Trade In
Get $170–$630 in credit when you trade in iPhone 12 or higher.
2
Get your estimate
Apple Card
Get up to 3% Daily Cash back with every purchase.
Learn more
Apply now
Apply now
Apple TV+
FAM Gallery
Watch now
Strong and Calm Comb

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [15]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [16]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [17]:
print(user_prompt_for(websiteDescription))

You are looking at a website titled Apple
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Apple
Apple
Store
Mac
iPad
iPhone
Watch
Vision
AirPods
TV & Home
Entertainment
Accessories
Support
0
+
iPhone
Meet the iPhone 16 family.
Learn more
Shop iPhone
Built for Apple Intelligence.
Buy Mac or iPad for college
with education savings
Choose AirPods or an eligible accessory
1
Shop
F1 The Movie
Starring Brad Pitt. In theaters June 27.
Watch the trailer
Watch the trailer
Get tickets
MacBook Air
Sky blue color.
Sky high performance with M4.
Learn more
Buy
Built for Apple Intelligence.
iPad Pro
Unbelievably thin. Incredibly powerful.
Learn more
Buy
Built for Apple Intelligence.
iPad Air
Now supercharged by the M3 chip.
Learn more
Buy
Built for Apple Intelligence.
Apple Podcasts
20 years of podcasts. 20 unforgettable shows.
Explore the list
Apple Trade In
Get $170–$630 in credi

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```python
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]
```
To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [18]:
messages = [
    {"role": "system", "content": "You are a fantastic and cunning assistant"},
    {"role": "user", "content": "What is the first wonder of the world?"}
]

In [19]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

The first of the Seven Wonders of the Ancient World is often considered to be the Great Pyramid of Giza. Located in Egypt, it is the only one of the seven wonders that still exists today. The pyramid was built as a tomb for the Pharaoh Khufu around 2580–2560 BC and is an extraordinary feat of engineering and construction.


## And now let's build useful messages for GPT-4o-mini, using a function

In [20]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [21]:
# Try this out, and then try for a few more websites

messages_for(websiteDescription)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Apple\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nApple\nApple\nStore\nMac\niPad\niPhone\nWatch\nVision\nAirPods\nTV & Home\nEntertainment\nAccessories\nSupport\n0\n+\niPhone\nMeet the iPhone 16 family.\nLearn more\nShop iPhone\nBuilt for Apple Intelligence.\nBuy Mac or\xa0iPad for\xa0college\nwith education savings\nChoose AirPods or an eligible accessory\n1\nShop\nF1 The Movie\nStarring Brad\xa0Pitt. In\xa0theaters\xa0June\xa027.\nWatch the trailer\nWatch the trailer\nGet tickets\nMacBook Air\nSky blue color.\nSky high performance with M4.\nLearn more\nBuy\nBuilt for Apple Intelligence.\niPad Pro\n

## Put the practice to test now and execute the function to summarize the content of a website

In [22]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [23]:
summarize("https://apple.com")

'# Apple Website Summary\n\nThe Apple website showcases its latest products, including the **iPhone 16 family**, **MacBook Air** in sky blue, and updated **iPad** models featuring the M3 and M4 chips. Users can explore the educational savings available for Mac and iPad purchases along with promotions for trading in eligible devices, offering credits ranging from $170 to $630.\n\n## Highlights:\n- **New Product Announcements**:\n  - **iPhone 16**: Introduced with enhancements leveraging Apple Intelligence.\n  - **MacBook Air**: Now available in a new sky blue color with high performance thanks to the M4 chip.\n  - **iPad Pro and iPad Air**: Both models boast incredible power, the latter being supercharged by the M3 chip.\n\n- **Apple Entertainment**:\n  - Promotion for **F1 The Movie** featuring Brad Pitt, with a trailer available and tickets on sale.\n  - Celebrates **20 years of podcasts** on Apple Podcasts with a curated list of shows.\n\n- **Apple Card Benefits**: Users can earn up 

In [24]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))
    #display(summary)

In [25]:
display_summary("https://apple.com")

# Apple Website Summary

The Apple website offers an overview of its product offerings, including the latest iPhone models, Mac, iPad, Apple Watch, and accessories. 

## Key Highlights:

- **iPhone 16 Family**: The website features the new iPhone 16 lineup, promoting its advanced capabilities.
  
- **Mac & iPad Promotions**: There are educational discounts available for students purchasing Macs or iPads alongside AirPods and eligible accessories.

- **New Product Features**:
  - **MacBook Air**: Introduced in a new sky blue color with superior performance powered by the M4 chip.
  - **iPad Pro & iPad Air**: Both models are highlighted for their thin design and increased power efficiency, with the iPad Air equipped with the M3 chip.

- **Apple Podcasts**: Celebrating 20 years of podcasts, the site mentions a curated list of 20 memorable shows.

- **Trade-In Program**: Users can trade in their iPhones from the 12 model or higher for credits ranging from $170 to $630, encouraging device upgrades.

- **Apple Card**: Offers up to 3% Daily Cash back on purchases, with specific application conditions noted.

- **Upcoming Entertainment Releases**:
  - Promotion for *F1 The Movie*, featuring Brad Pitt, scheduled for release on June 27.

This summary highlights the main products and promotions without focusing on navigation elements or customer service sections.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [26]:
display_summary("https://spotify.com")

# Spotify - Web Player: Music for Everyone

Spotify's Web Player offers users a convenient platform to listen to music across various genres, playlists, and podcasts. The service is accessible from any web browser, allowing users to enjoy their favorite tracks without needing to download additional software. The website emphasizes user-friendly navigation and personalized music recommendations, making it simple for everyone to discover new content.

## Key Features:
- **Diverse Music Library**: Access to millions of songs and playlists.
- **Custom Playlists**: Users can create and share their playlists.
- **Podcasts**: A wide selection of podcasts available for streaming.
- **Personalization**: Tailored music recommendations based on user preferences.

No specific news or announcements were highlighted on the website at the time of the analysis.

In [27]:
display_summary("https://www.cloudflare.com/")

# Summary of "Access Denied" Website

The website indicates that access to "http://www2.hm.com/" is restricted, and users do not have permission to view its contents on the server. A reference number is provided for tracking the error.

## Key Points:
- Users are denied access to the specified site.
- A reference code is included for further inquiries.
- No news or announcements are present due to the access restriction.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise, you experienced calling the Cloud API of a Frontier Model (a leading model at the frontier of AI) for the first time. We will be using APIs like OpenAI at many stages in the course, in addition to building our own LLMs.

More specifically, we've applied this to Summarization - a classic Gen AI use case to make a summary. This can be applied to any business vertical - summarizing the news, summarizing financial performance, summarizing a resume in a cover letter - the applications are limitless. Consider how you could apply Summarization in your business, and try prototyping a solution.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue - now try yourself</h2>
            <span style="color:#900;">Use the cell below to make your own simple commercial example. Stick with the summarization use case for now. Here's an idea: write something that will take the contents of an email, and will suggest an appropriate short subject line for the email. That's the kind of feature that might be built into a commercial email tool.</span>
        </td>
    </tr>
</table>

In [ ]:
# Step 1: Create your prompts

system_prompt = "something here"
user_prompt = """
    Lots of text
    Can be pasted here
"""

# Step 2: Make the messages list

messages = [] # fill this in

# Step 3: Call OpenAI

response =

# Step 4: print the result

print(

## An extra exercise for those who enjoy web scraping

You may notice that if you try `display_summary("https://openai.com")` - it doesn't work! That's because OpenAI has a fancy website that uses Javascript. There are many ways around this that some of you might be familiar with. For example, Selenium is a hugely popular framework that runs a browser behind the scenes, renders the page, and allows you to query it. If you have experience with Selenium, Playwright or similar, then feel free to improve the Website class to use them. In the community-contributions folder, you'll find an example Selenium solution from a student (thank you!)

# Sharing your code

I'd love it if you share your code afterwards so I can share it with others! You'll notice that some students have already made changes (including a Selenium implementation) which you will find in the community-contributions folder. If you'd like add your changes to that folder, submit a Pull Request with your new versions in that folder and I'll merge your changes.

If you're not an expert with git (and I am not!) then GPT has given some nice instructions on how to submit a Pull Request. It's a bit of an involved process, but once you've done it once it's pretty clear. As a pro-tip: it's best if you clear the outputs of your Jupyter notebooks (Edit >> Clean outputs of all cells, and then Save) for clean notebooks.

Here are good instructions courtesy of an AI friend:  
https://chatgpt.com/share/677a9cb5-c64c-8012-99e0-e06e88afd293